This the tutorial that can be found at http://machinelearningmastery.com/tutorial-first-neural-network-python-keras/.
I thought it more practical to have a realistic dataset, than the more common *handwritten digits* or *cats & dogs* datasets.

The objective is to build a basic model to predict whether or not the patients will have diabetes over the next 5 years.
The required dataset has been downloaded to the `datasets/uci` folder during VM setup.

Both `TensorFlow` and `Theano` are installed. This notebook uses `Theano`.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [29]:
# Load the dataset. The 9th column is the target
dataset = numpy.loadtxt("../datasets/uci/pima-indians-diabetes.csv", delimiter=",")
X = dataset[:,0:8]
Y = dataset[:,8]

In [30]:
# Create the model
# Please refer to the above mentioned URL for explanations.
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

/home/vagrant/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform")`
/home/vagrant/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
/home/vagrant/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`


In [31]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# Fit the model
model.fit(X, Y, epochs=200, batch_size=10)

Epoch 1/200
768/768 [==============================] - 0s - loss: 0.6771 - acc: 0.6510     
Epoch 2/200
768/768 [==============================] - 0s - loss: 0.6598 - acc: 0.6510     
Epoch 3/200
768/768 [==============================] - 0s - loss: 0.6484 - acc: 0.6510     
Epoch 4/200
768/768 [==============================] - 0s - loss: 0.6402 - acc: 0.6510     
Epoch 5/200
768/768 [==============================] - 0s - loss: 0.6328 - acc: 0.6510     
Epoch 6/200
768/768 [==============================] - 0s - loss: 0.6196 - acc: 0.6510     
Epoch 7/200
768/768 [==============================] - 0s - loss: 0.6192 - acc: 0.6510     
Epoch 8/200
768/768 [==============================] - 0s - loss: 0.6144 - acc: 0.6510     
Epoch 9/200
768/768 [==============================] - 0s - loss: 0.6092 - acc: 0.6510     
Epoch 10/200
768/768 [==============================] - 0s - loss: 0.6161 - acc: 0.6510     
Epoch 11/200
768/768 [==============================] - 0s - loss: 0.6057 - acc

In [47]:
# Evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.3f%%" % (model.metrics_names[1], scores[1]*100))

 32/768 [>.............................] - ETA: 0sacc: 79.427%


## Part 2 - Using Pickle and Shoe dataset

Information adapted from [source](https://doyleax.github.io/Portfolio/capstone.html)

---

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pickle
import matplotlib.pyplot as plt
np.random.seed(7)

%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('shoenumbers.pickle', 'rb') as handle:
    X = pickle.load(handle, encoding='bytes')
with open('targetvars.pickle', 'rb') as handle:
    y = pickle.load(handle, encoding='bytes')

In [3]:
print(np.array(X).shape)
print(np.array(y).shape)

(1780, 200, 280, 3)
(1780,)


In [4]:
baseline = y.mean()
print(baseline)

0.502247191011


In [5]:
# Change size of images so it's not so huge when we pass it through keras
import cv2
new_X = []
for row in range(len(X)):
    new_X.append(cv2.resize(X[row], (140,100)))
np.array(new_X).shape

(1780, 100, 140, 3)

In [6]:
# Prepare data for training and testing
X = np.array(new_X)
X = X.astype('float32')
X /= 255.0
y = np.array(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [30]:
# Model creation by machinelearningmastery
num_classes=2
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100,140,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [31]:
# Compile the model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

Used https://github.com/fchollet/keras/issues/3109 to figure out that sparse_categorial_crossentropy was the loss function needed.

In [26]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 1335 samples, validate on 445 samples
Epoch 1/25
1335/1335 [==============================] - 103s - loss: 7.8471 - acc: 0.5019 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 2/25
1335/1335 [==============================] - 111s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 3/25
1335/1335 [==============================] - 116s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 4/25
1335/1335 [==============================] - 109s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 5/25
1335/1335 [==============================] - 107s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 6/25
1335/1335 [==============================] - 110s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 7/25
1335/1335 [==============================] - 111s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 8/25
1335/1335 [==============================] - 113s - l

In [27]:
y_pred = model.predict_classes(X_test)

445/445 [==============================] - 6s     


In [29]:
y_pred.sum()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,